<a href="https://colab.research.google.com/github/LinarKulinar/EnterpriseDataBase/blob/main/Lab_2_EnterpriseDataBase_6133_Akhmetov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Корпоративные базы данных**

# **Лабораторная работа 2**

Выполнил: 6133, Ахметов Линар

In [1]:
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
import sklearn.model_selection
from sklearn.model_selection import train_test_split
import sklearn.metrics as metr
from sklearn.metrics import classification_report
import sklearn.pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier

Данные представляет размеченый датасет с различными генетическими мутациями.

Скачать можно на [kaggle](https://www.kaggle.com/datasets/kevinarvai/clinvar-conflicting).

In [2]:
df = pd.read_csv('clinvar_conflicting.csv', dtype={0: object, 38: str, 40: object})
df.head()

,CHROM,POS,REF,ALT,AF_ESP,AF_EXAC,AF_TGP,CLNDISDB,CLNDISDBINCL,CLNDN,...,SIFT,PolyPhen,MOTIF_NAME,MOTIF_POS,HIGH_INF_POS,MOTIF_SCORE_CHANGE,LoFtool,CADD_PHRED,CADD_RAW,BLOSUM62
0,1,1168180,G,C,0.0771,0.10020,0.1066,MedGen:CN169374,NaN,not_specified,...,tolerated,benign,NaN,NaN,NaN,NaN,NaN,1.053,-0.208682,2.0
1,1,1470752,G,A,0.0000,0.00000,0.0000,"MedGen:C1843891,OMIM:607454,Orphanet:ORPHA9877...",NaN,Spinocerebellar_ataxia_21|not_provided,...,deleterious_low_confidence,benign,NaN,NaN,NaN,NaN,NaN,31.000,6.517838,-3.0
2,1,1737942,A,G,0.0000,0.00001,0.0000,"Human_Phenotype_Ontology:HP:0000486,MedGen:C00...",NaN,Strabismus|Nystagmus|Hypothyroidism|Intellectu...,...,deleterious,probably_damaging,NaN,NaN,NaN,NaN,NaN,28.100,6.061752,-1.0
3,1,2160305,G,A,0.0000,0.00000,0.0000,"MedGen:C1321551,OMIM:182212,SNOMED_CT:83092002...",NaN,Shprintzen-Goldberg_syndrome|not_provided,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.500,3.114491,NaN
4,1,2160305,G,T,0.0000,0.00000,0.0000,"MedGen:C1321551,OMIM:182212,SNOMED_CT:83092002",NaN,Shprintzen-Goldberg_syndrome,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.700,4.766224,-3.0


# Этап 2. Подготовка данных

In [3]:
df = df.drop(["CLNDN",'CLNDISDBINCL','MOTIF_NAME','MOTIF_POS','HIGH_INF_POS','MOTIF_SCORE_CHANGE','DISTANCE','CLNHGVS','CLNDISDB',"CLNDNINCL",'CLNSIGINCL','SSR','INTRON','Codons','MC','CLNVI','SYMBOL','Amino_acids','Protein_position','CDS_position','EXON','cDNA_position','Feature', 'Feature_type','BAM_EDIT'], axis=1)
df.head()

,CHROM,POS,REF,ALT,AF_ESP,AF_EXAC,AF_TGP,CLNVC,ORIGIN,CLASS,...,Consequence,IMPACT,BIOTYPE,STRAND,SIFT,PolyPhen,LoFtool,CADD_PHRED,CADD_RAW,BLOSUM62
0,1,1168180,G,C,0.0771,0.10020,0.1066,single_nucleotide_variant,1,0,...,missense_variant,MODERATE,protein_coding,1.0,tolerated,benign,NaN,1.053,-0.208682,2.0
1,1,1470752,G,A,0.0000,0.00000,0.0000,single_nucleotide_variant,1,0,...,missense_variant,MODERATE,protein_coding,-1.0,deleterious_low_confidence,benign,NaN,31.000,6.517838,-3.0
2,1,1737942,A,G,0.0000,0.00001,0.0000,single_nucleotide_variant,35,1,...,missense_variant,MODERATE,protein_coding,-1.0,deleterious,probably_damaging,NaN,28.100,6.061752,-1.0
3,1,2160305,G,A,0.0000,0.00000,0.0000,single_nucleotide_variant,33,0,...,missense_variant,MODERATE,protein_coding,1.0,NaN,NaN,NaN,22.500,3.114491,NaN
4,1,2160305,G,T,0.0000,0.00000,0.0000,single_nucleotide_variant,33,0,...,missense_variant,MODERATE,protein_coding,1.0,NaN,NaN,NaN,24.700,4.766224,-3.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65188 entries, 0 to 65187
Data columns (total 21 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   CHROM        65188 non-null  object 
 1   POS          65188 non-null  int64  
 2   REF          65188 non-null  object 
 3   ALT          65188 non-null  object 
 4   AF_ESP       65188 non-null  float64
 5   AF_EXAC      65188 non-null  float64
 6   AF_TGP       65188 non-null  float64
 7   CLNVC        65188 non-null  object 
 8   ORIGIN       65188 non-null  int64  
 9   CLASS        65188 non-null  int64  
 10  Allele       65188 non-null  object 
 11  Consequence  65188 non-null  object 
 12  IMPACT       65188 non-null  object 
 13  BIOTYPE      65172 non-null  object 
 14  STRAND       65174 non-null  float64
 15  SIFT         24836 non-null  object 
 16  PolyPhen     24796 non-null  object 
 17  LoFtool      60975 non-null  float64
 18  CADD_PHRED   64096 non-null  float64
 19  CADD

### Кодируем категориальные признаки с помощью `pd.factorize`

In [5]:
df['CHROM'] = pd.factorize(df['CHROM'])[0]
df['IMPACT'] = pd.factorize(df['IMPACT'])[0]
df['BIOTYPE'] = pd.factorize(df['BIOTYPE'])[0]
df['CLNVC'] = pd.factorize(df['CLNVC'])[0]
df['PolyPhen'] = pd.factorize(df['PolyPhen'])[0]
df['Consequence'] = pd.factorize(df['Consequence'])[0]
df['REF'] = pd.factorize(df['REF'])[0]
df['ALT'] = pd.factorize(df['ALT'])[0]
df['SIFT'] = pd.factorize(df['SIFT'])[0]
df['Allele'] = pd.factorize(df['Allele'])[0]

In [6]:
df.head(3)

,CHROM,POS,REF,ALT,AF_ESP,AF_EXAC,AF_TGP,CLNVC,ORIGIN,CLASS,...,Consequence,IMPACT,BIOTYPE,STRAND,SIFT,PolyPhen,LoFtool,CADD_PHRED,CADD_RAW,BLOSUM62
0,0,1168180,0,0,0.0771,0.10020,0.1066,0,1,0,...,0,0,0,1.0,0,0,NaN,1.053,-0.208682,2.0
1,0,1470752,0,1,0.0000,0.00000,0.0000,0,1,0,...,0,0,0,-1.0,1,0,NaN,31.000,6.517838,-3.0
2,0,1737942,1,2,0.0000,0.00001,0.0000,0,35,1,...,0,0,0,-1.0,2,1,NaN,28.100,6.061752,-1.0


In [7]:
df = df.fillna(0)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65188 entries, 0 to 65187
Data columns (total 21 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   CHROM        65188 non-null  int64  
 1   POS          65188 non-null  int64  
 2   REF          65188 non-null  int64  
 3   ALT          65188 non-null  int64  
 4   AF_ESP       65188 non-null  float64
 5   AF_EXAC      65188 non-null  float64
 6   AF_TGP       65188 non-null  float64
 7   CLNVC        65188 non-null  int64  
 8   ORIGIN       65188 non-null  int64  
 9   CLASS        65188 non-null  int64  
 10  Allele       65188 non-null  int64  
 11  Consequence  65188 non-null  int64  
 12  IMPACT       65188 non-null  int64  
 13  BIOTYPE      65188 non-null  int64  
 14  STRAND       65188 non-null  float64
 15  SIFT         65188 non-null  int64  
 16  PolyPhen     65188 non-null  int64  
 17  LoFtool      65188 non-null  float64
 18  CADD_PHRED   65188 non-null  float64
 19  CADD

### С помощью `sklearn.preprocessing.StandardScaler` нормализовываем значения признаков и строим для нормализовапнного датасета `KNeighborsClassifier` (без использования конвееров)

In [8]:
scaler = StandardScaler()

In [9]:
df_little=df[:65000:65]
df_little

,CHROM,POS,REF,ALT,AF_ESP,AF_EXAC,AF_TGP,CLNVC,ORIGIN,CLASS,...,Consequence,IMPACT,BIOTYPE,STRAND,SIFT,PolyPhen,LoFtool,CADD_PHRED,CADD_RAW,BLOSUM62
0,0,1168180,0,0,0.0771,0.10020,0.1066,0,1,0,...,0,0,0,1.0,0,0,0.00000,1.053,-0.208682,2.0
65,0,19201919,2,0,0.0676,0.03354,0.0655,0,1,0,...,0,0,0,-1.0,2,0,0.91500,24.100,4.394493,0.0
130,0,43804340,0,1,0.0238,0.02504,0.0190,0,1,0,...,0,0,0,1.0,0,2,0.34300,5.142,0.244588,1.0
195,0,94466624,3,3,0.0070,0.00187,0.0046,0,1,1,...,0,0,0,-1.0,2,1,0.01070,35.000,7.983524,0.0
260,0,100340787,0,1,0.0024,0.04757,0.0797,0,1,0,...,0,0,0,1.0,0,0,0.03570,0.030,-0.867406,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64675,22,131212944,1,2,0.0983,0.07222,0.0474,0,1,0,...,5,2,0,-1.0,-1,-1,0.10700,0.006,-1.204650,0.0
64740,22,136651194,0,3,0.0005,0.00009,0.0003,0,1,1,...,5,2,0,1.0,-1,-1,0.00000,9.921,0.875480,0.0
64805,22,152960025,3,3,0.0000,0.00000,0.0000,0,1,0,...,5,2,0,1.0,-1,-1,0.24100,13.510,1.536023,0.0
64870,22,153218362,3,3,0.0329,0.01514,0.0347,0,1,0,...,5,2,0,-1.0,-1,-1,0.00303,11.190,1.095208,0.0


In [10]:
X = df_little.drop(columns=['CLASS'])
y = df_little.CLASS
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state=42)

In [11]:
#нормируем данные:
scaler.fit(X_train) 
tdtr = scaler.transform(X_train)
tdt = scaler.transform(X_test)

# Этап 3 и 4. Выбор классификатора и определение оптимальных значениев гиперпараметров

# Классификатор `KNN`

In [12]:
clf = KNeighborsClassifier(n_neighbors=5) 
clf.fit(tdtr, y_train)
cl1 = clf.predict(tdt)

In [13]:
#доля правильных ответов:
metr.accuracy_score(y_test, cl1) 

0.696

### Для `KNN` подбираем гиперпараметры с помощью `GridSearchCV`

In [14]:
knn = sklearn.neighbors.KNeighborsClassifier(n_jobs=-1)

knn_params = { 'n_neighbors': range(3, 17) } 

knn_grid = sklearn.model_selection.GridSearchCV(knn, knn_params) 

knn_grid.fit(tdtr, y_train)

GridSearchCV(estimator=KNeighborsClassifier(n_jobs=-1),
             param_grid={'n_neighbors': range(3, 17)})

In [15]:
# получаем оптимальные параметры для заданного классификатора:
knn_grid.best_params_, knn_grid.best_score_ 

({'n_neighbors': 14}, 0.74)

In [16]:
# доля правильных ответов:
metr.accuracy_score(y_test, knn_grid.predict(tdt)) 

0.724

# Классификатор `DecisionTreeClassifier`

In [17]:
clf_tree = DecisionTreeClassifier( 
    criterion = 'entropy', max_depth = 3, random_state = 42
)
clf_tree.fit(tdtr, y_train)
cl_t = clf_tree.predict(tdt)
metr.accuracy_score(y_test, cl_t)

0.732

In [18]:
print("Classification Report :\n ", classification_report(cl_t, y_test))

Classification Report :
                precision    recall  f1-score   support

           0       1.00      0.73      0.85       250
           1       0.00      0.00      0.00         0

    accuracy                           0.73       250
   macro avg       0.50      0.37      0.42       250
weighted avg       1.00      0.73      0.85       250



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Для `DecisionTreeClassifier` подбираем гиперпараметры с помощью `GridSearchCV` (перебираем `max_depth`, `max_features`)

In [19]:
tree = DecisionTreeClassifier(random_state=42)
tree_params = {'max_depth': range(1, 25),
              'max_features': range(2, 20)}

tree_grid = GridSearchCV(tree, tree_params) # подбираем параметры для дерева

tree_grid.fit( tdtr, y_train )

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'max_depth': range(1, 25),
                         'max_features': range(2, 20)})

In [20]:
# оптимальные параметры:
tree_grid.best_params_, tree_grid.best_score_ 

({'max_depth': 3, 'max_features': 7}, 0.7466666666666667)

In [21]:
# доля правильных ответов:
metr.accuracy_score(y_test, tree_grid.predict(tdt)) 

0.736

# Классификатор `RandomForestClassifier`

In [22]:
rf_clf = RandomForestClassifier(n_estimators=8, random_state=42)
rf_clf.fit(tdtr, y_train)

RandomForestClassifier(n_estimators=8, random_state=42)

In [23]:
metr.accuracy_score(y_test, rf_clf.predict(tdt))

0.728

### Для `RandomForestClassifier` подбираем гиперпараметры с помощью `GridSearchCV` (перебираем `max_depth`, `max_features`)

In [24]:
forest_params = {
    'max_depth': range(1,24),
    'max_features': range(3,15)
}

forest_grid = sklearn.model_selection.GridSearchCV(rf_clf, forest_params, cv=5)

forest_grid.fit(tdtr, y_train)

GridSearchCV(cv=5,
             estimator=RandomForestClassifier(n_estimators=8, random_state=42),
             param_grid={'max_depth': range(1, 24),
                         'max_features': range(3, 15)})

In [25]:
#оптимальные параметры
forest_grid.best_params_, forest_grid.best_score_ 

({'max_depth': 3, 'max_features': 9}, 0.7466666666666667)

In [26]:
# доля правильных ответов:
metr.accuracy_score(y_test, forest_grid.predict(tdt)) 

0.728

# Этап 5. Выбор классификатора с учетом гиперпараметров

In [27]:
# доля правильных ответов:
metr.accuracy_score(y_test, knn_grid.predict(tdt)) 

0.724

In [28]:
# доля правильных ответов:
metr.accuracy_score(y_test, tree_grid.predict(tdt)) 

0.736

In [29]:
# доля правильных ответов:
metr.accuracy_score(y_test, forest_grid.predict(tdt)) 

0.728

Классификатором с наилучшим `accuracy=0.736` оказалось решающее дерево с нижеприведенными параметрами.

In [30]:
tree_grid.best_params_, tree_grid.best_score_ 

({'max_depth': 3, 'max_features': 7}, 0.7466666666666667)